In [1]:
from PIL import Image  # has to be a separate cell to avoid import errors

In [2]:
from utils.data_loading import load_cifar10

train_data = load_cifar10(train=True, raw=True, return_loader=False)

In [5]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from collections import defaultdict
import random

# pick one random index for each class
class_to_indices = defaultdict(list)
for i in range(len(train_data)):
    _, label = train_data[i]
    class_to_indices[label].append(i)

# select 1 random image per class
one_per_class = [random.choice(indices) for indices in class_to_indices.values()]

# limit to 10 images
if len(one_per_class) > 10:
    one_per_class = random.sample(one_per_class, 10)

n = len(one_per_class)
cols = 5  # e.g. 5x2 grid for 10 images
rows = int(np.ceil(n / cols))

fig = make_subplots(
    rows=rows,
    cols=cols,
    subplot_titles=[train_data.classes[train_data[i][1]] for i in one_per_class],
)

for idx, img_idx in enumerate(one_per_class):
    row = idx // cols + 1
    col = idx % cols + 1

    img = train_data[img_idx][0].numpy()  # [C,H,W]
    img = np.transpose(img, (1, 2, 0))  # -> [H,W,C]
    img = (img * 255).astype(np.uint8)  # scale to [0,255]

    fig.add_trace(go.Image(z=img), row=row, col=col)

fig.update_layout(width=cols * 320, height=rows * 320, showlegend=False)

fig.show()


In [12]:
import torch

counts = torch.bincount(torch.tensor(train_data.targets, dtype=torch.int64))
for label, count in enumerate(counts):
    print(
        f"Label {label}: {count.item()} samples, {count.item() / len(train_data) * 100:.2f}% of dataset"
    )


Label 0: 5000 samples, 10.00% of dataset
Label 1: 5000 samples, 10.00% of dataset
Label 2: 5000 samples, 10.00% of dataset
Label 3: 5000 samples, 10.00% of dataset
Label 4: 5000 samples, 10.00% of dataset
Label 5: 5000 samples, 10.00% of dataset
Label 6: 5000 samples, 10.00% of dataset
Label 7: 5000 samples, 10.00% of dataset
Label 8: 5000 samples, 10.00% of dataset
Label 9: 5000 samples, 10.00% of dataset


In [6]:
import torch

# train_data already loaded with transforms.ToTensor() but NO Normalize
mean = torch.zeros(3)
std = torch.zeros(3)
n_samples = 0

for img, _ in train_data:
    # img is [C, H, W] float in [0,1]
    n_pixels = img.numel() // 3  # H*W
    mean += img.view(3, -1).mean(dim=1)
    std += img.view(3, -1).std(dim=1)
    n_samples += 1

mean /= n_samples
std /= n_samples

print("mean:", mean)
print("std:", std)


mean: tensor([0.4914, 0.4822, 0.4465])
std: tensor([0.2023, 0.1994, 0.2010])
